In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [2]:
%config Completer.use_jedi = False

# Importing Database from csv of Jakarta Postal Codes

In [3]:
df = pd.read_csv('JakartaPostalCodes.csv')

In [4]:
df.head()

,Kode Pos,Kecamatan,Kelurahan
0,12430,Cilandak,Cilandak Barat
1,12410,Cilandak,Cipete Selatan
2,12420,Cilandak,Gandaria Selatan
3,12440,Cilandak,Lebak Bulus
4,12450,Cilandak,Pondok Labu


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Kode Pos   65 non-null     int64 
 1   Kecamatan  65 non-null     object
 2   Kelurahan  65 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.6+ KB


In [6]:
len(df['Kelurahan'])

65

# Now lets Search each Latitude and Longitude of each neighborhood

In [7]:
geolocator = Nominatim(user_agent="jakartans_users")
location = geolocator.geocode("Jakarta Selatan")
print(location.latitude, location.longitude)
latjaksel = location.latitude
longjaksel = location.longitude


-6.28381815 106.80486324917382


In [8]:
longlat = []
for x in range(len(df['Kelurahan'])):
    location = geolocator.geocode("{},{}".format(df['Kelurahan'][x],df['Kecamatan'][x]))
    longlat.append([location.latitude, location.longitude])


In [9]:
longlat = np.array(longlat)


# As we finished getting our latitude and longitude for each neighborhood, its time to add it into dataframe

In [10]:
longlat = pd.DataFrame(longlat,columns=['Latitude','Longitude'])

In [11]:
longlat.head()

,Latitude,Longitude
0,-6.290266,106.796215
1,-6.272540,106.805768
2,-6.271562,106.795172
3,-6.302568,106.780354
4,-6.309144,106.797938


In [12]:
df = pd.concat([df,longlat],axis=1)

In [13]:
df.head()

,Kode Pos,Kecamatan,Kelurahan,Latitude,Longitude
0,12430,Cilandak,Cilandak Barat,-6.290266,106.796215
1,12410,Cilandak,Cipete Selatan,-6.272540,106.805768
2,12420,Cilandak,Gandaria Selatan,-6.271562,106.795172
3,12440,Cilandak,Lebak Bulus,-6.302568,106.780354
4,12450,Cilandak,Pondok Labu,-6.309144,106.797938


# Create a map of Jakarta

In [14]:
map_jkt = folium.Map(location=[latjaksel,longjaksel], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Kelurahan']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_jkt)  
    
map_jkt

## This is where we would like to gain information from foursquare API

In [15]:
CLIENT_ID = '2VCVAGIT4UT1BTHDH5O4J0HNCAB0TYOVZOCXCWGL3QNHE0JF' # your Foursquare ID
CLIENT_SECRET = '1PEFZOOG0J23BMA4MZ1P0HR05NQSDITX451QVLCB2D0VGXRG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2VCVAGIT4UT1BTHDH5O4J0HNCAB0TYOVZOCXCWGL3QNHE0JF
CLIENT_SECRET:1PEFZOOG0J23BMA4MZ1P0HR05NQSDITX451QVLCB2D0VGXRG


## Exploring Neighborhood in  South Jakarta (Jaksel)

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### with the function above, we can now get nearby venues for all neighborhood

In [17]:
jaksel_venues = getNearbyVenues(names=df['Kelurahan'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'])

Cilandak Barat
Cipete Selatan
Gandaria Selatan
Lebak Bulus
Pondok Labu
Ciganjur
Cipedak
Jagakarsa
Lenteng Agung
Srengseng Sawah
Tanjung Barat
Cipete Utara
Gandaria Utara
Gunung
Kramat Pela
Melawai
Petogogan
Pulo
Rawa Barat
Selong
Senayan
Cipulir
Grogol Selatan
Grogol Utara
Kebayoran Lama Selatan
Kebayoran Lama Utara
Pondok Pinang
Bangka
Kuningan Barat
Mampang Prapatan
Pela Mampang
Tegal Parang
Cikoko
Duren Tiga
Kalibata
Pancoran
Pengadegan
Rawajati
Cilandak Timur
Jatipadang
Kebagusan
Pasar Minggu
Pejaten Barat
Pejaten Timur
Ragunan
Bintaro
Pesanggrahan
Petukangan Selatan
Petukangan Utara
Ulujami
Guntur
Karet
Karet Kuningan
Karet Semanggi
Kuningan Timur
Menteng Atas
Pasar Manggis
Setiabudi
Bukit Duri
Kebon Baru
Manggarai
Manggarai Selatan
Menteng Dalam
Tebet Barat
Tebet Timur


In [18]:
print(jaksel_venues.shape)
jaksel_venues.head()

(1057, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Cilandak Barat,-6.290266,106.796215,Baskin-Robbins,-6.291314,106.799668,Ice Cream Shop
1,Cilandak Barat,-6.290266,106.796215,Domino's Pizza Fatmawati,-6.291766,106.796075,Pizza Place
2,Cilandak Barat,-6.290266,106.796215,TOUS les JOURS,-6.291655,106.799730,Bakery
3,Cilandak Barat,-6.290266,106.796215,Bebek Tepi Sawah,-6.291700,106.799664,Asian Restaurant
4,Cilandak Barat,-6.290266,106.796215,Domino's Pizza,-6.289279,106.795409,Pizza Place


In [19]:
jaksel_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bangka,6,6,6,6,6,6
Bintaro,18,18,18,18,18,18
Ciganjur,4,4,4,4,4,4
Cikoko,12,12,12,12,12,12
Cilandak Barat,56,56,56,56,56,56
Cilandak Timur,6,6,6,6,6,6
Cipedak,5,5,5,5,5,5
Cipete Selatan,5,5,5,5,5,5
Cipete Utara,4,4,4,4,4,4


finding out unique categories

In [20]:
print('There are {} uniques categories.'.format(len(jaksel_venues['Venue Category'].unique())))

There are 154 uniques categories.


## Analyzing each neighborhood 

In [21]:
# one hot encoding
jaksel_onehot = pd.get_dummies(jaksel_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
jaksel_onehot['Neighborhood'] = jaksel_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [jaksel_onehot.columns[-1]] + list(jaksel_onehot.columns[:-1])
jaksel_onehot = jaksel_onehot[fixed_columns]

jaksel_onehot.head()

,Wings Joint,ATM,Acehnese Restaurant,Airport,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Betawinese Restaurant,Bistro,Bookstore,Boutique,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Line,Bus Stop,Butcher,Café,Chinese Restaurant,Clothing Store,Coffee Shop,College Residence Hall,Comfort Food Restaurant,Convenience Store,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Donut Shop,Dutch Restaurant,Eastern European Restaurant,Electronics Store,Embassy / Consulate,Event Space,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,High School,Hobby Shop,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Meatball Place,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Javanese Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Manadonese Restaurant,Market,Massage Studio,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Moving Target,Multiplex,Music School,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,Office,Optical Shop,Padangnese Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Post Office,Pub,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shopping Mall,Snack Place,Soccer Field,Soup Place,Spa,Spanish Restaurant,Sports Bar,Sports Club,Steakhouse,Sundanese Restaurant,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theme Park,Toy / Game Store,Train Station,Travel Agency,Vape Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Cilandak Barat,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Cilandak Barat,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Cilandak Barat,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Cilandak Barat,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Cilandak Barat,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
jaksel_onehot.shape

(1057, 154)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [23]:
jaksel_grouped = jaksel_onehot.groupby('Neighborhood').mean().reset_index()
jaksel_grouped

,Neighborhood,Wings Joint,ATM,Acehnese Restaurant,Airport,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Betawinese Restaurant,Bistro,Bookstore,Boutique,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Line,Bus Stop,Butcher,Café,Chinese Restaurant,Clothing Store,Coffee Shop,College Residence Hall,Comfort Food Restaurant,Convenience Store,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Donut Shop,Dutch Restaurant,Eastern European Restaurant,Electronics Store,Embassy / Consulate,Event Space,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,High School,Hobby Shop,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Meatball Place,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Javanese Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Manadonese Restaurant,Market,Massage Studio,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Moving Target,Multiplex,Music School,Music Store,Music Venue,Nightclub,Noodle House,Office,Optical Shop,Padangnese Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Post Office,Pub,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shopping Mall,Snack Place,Soccer Field,Soup Place,Spa,Spanish Restaurant,Sports Bar,Sports Club,Steakhouse,Sundanese Restaurant,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theme Park,Toy / Game Store,Train Station,Travel Agency,Vape Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar
0,Bangka,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.500000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bintaro,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.0,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.055556,0.055556,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.222222,0.055556,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.

#### printing top common venues

In [24]:
num_top_venues = 5

for hood in jaksel_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = jaksel_grouped[jaksel_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bangka----
                   venue  freq
0   Fast Food Restaurant  0.50
1  Indonesian Restaurant  0.33
2                 Bakery  0.17
3            Pizza Place  0.00
4           Optical Shop  0.00


----Bintaro----
                   venue  freq
0   Fast Food Restaurant  0.22
1  Indonesian Restaurant  0.17
2      Convenience Store  0.17
3            Coffee Shop  0.06
4      Fish & Chips Shop  0.06


----Ciganjur----
               venue  freq
0         Restaurant  0.25
1       Soccer Field  0.25
2  Convenience Store  0.25
3               Café  0.25
4        Wings Joint  0.00


----Cikoko----
                  venue  freq
0  Fast Food Restaurant  0.17
1     Convenience Store  0.17
2           Coffee Shop  0.17
3           Gas Station  0.08
4   Japanese Restaurant  0.08


----Cilandak Barat----
                   venue  freq
0            Coffee Shop  0.14
1                   Café  0.09
2       Asian Restaurant  0.09
3  Indonesian Restaurant  0.07
4            Pizza Place  0.05


----

                   venue  freq
0            Coffee Shop  0.17
1             Food Court  0.07
2           Noodle House  0.07
3  Indonesian Restaurant  0.07
4   Fast Food Restaurant  0.05


----Selong----
                 venue  freq
0  Japanese Restaurant  0.14
1          Coffee Shop  0.14
2     Sushi Restaurant  0.09
3                Hotel  0.09
4    Convenience Store  0.06


----Senayan----
                   venue  freq
0    Japanese Restaurant  0.10
1            Coffee Shop  0.08
2      Korean Restaurant  0.07
3  Indonesian Restaurant  0.06
4       Department Store  0.05


----Setiabudi----
                venue  freq
0   Convenience Store  0.12
1        Noodle House  0.09
2  Italian Restaurant  0.06
3                Café  0.06
4         Pizza Place  0.06


----Srengseng Sawah----
                    venue  freq
0  College Residence Hall  0.33
1      Light Rail Station  0.33
2              Theme Park  0.33
3             Wings Joint  0.00
4   Padangnese Restaurant  0.00


----Tanjung

### Great! we have find the top 5 common venues for each neighborhood lets put it into dataframe

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = jaksel_grouped['Neighborhood']

for ind in np.arange(jaksel_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(jaksel_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bangka,Fast Food Restaurant,Indonesian Restaurant,Bakery,Pizza Place,Optical Shop,Music School,Music Store,Music Venue,Nightclub,Noodle House
1,Bintaro,Fast Food Restaurant,Indonesian Restaurant,Convenience Store,Coffee Shop,Fish & Chips Shop,Asian Restaurant,Gym,Bakery,Sandwich Place,Supermarket
2,Ciganjur,Restaurant,Soccer Field,Convenience Store,Café,Wings Joint,Padangnese Restaurant,Music Venue,Nightclub,Noodle House,Office
3,Cikoko,Fast Food Restaurant,Convenience Store,Coffee Shop,Gas Station,Japanese Restaurant,Asian Restaurant,Supermarket,Café,Chinese Restaurant,Padangnese Restaurant
4,Cilandak Barat,Coffee Shop,Café,Asian Restaurant,Indonesian Restaurant,Pizza Place,Fast Food Restaurant,Ice Cream Shop,Bookstore,Bakery,Thai Restaurant


In [27]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bangka,Fast Food Restaurant,Indonesian Restaurant,Bakery,Pizza Place,Optical Shop,Music School,Music Store,Music Venue,Nightclub,Noodle House
1,Bintaro,Fast Food Restaurant,Indonesian Restaurant,Convenience Store,Coffee Shop,Fish & Chips Shop,Asian Restaurant,Gym,Bakery,Sandwich Place,Supermarket
2,Ciganjur,Restaurant,Soccer Field,Convenience Store,Café,Wings Joint,Padangnese Restaurant,Music Venue,Nightclub,Noodle House,Office
3,Cikoko,Fast Food Restaurant,Convenience Store,Coffee Shop,Gas Station,Japanese Restaurant,Asian Restaurant,Supermarket,Café,Chinese Restaurant,Padangnese Restaurant
4,Cilandak Barat,Coffee Shop,Café,Asian Restaurant,Indonesian Restaurant,Pizza Place,Fast Food Restaurant,Ice Cream Shop,Bookstore,Bakery,Thai Restaurant
5,Cilandak Timur,Coffee Shop,Hot Dog Joint,Food Court,Indonesian Restaurant,Pub,Pizza Place,Pharmacy,Pet Store,Performing Arts Venue,Park
6,Cipedak,Food,Department Store,Grocery Store,Farmers Market,Fast Food Restaurant,Wings Joint,Nightclub,Noodle House,Office,Optical Shop
7,Cipete Selatan,Moving Target,Javanese Restaurant,Arcade,Food Court,Café,Wings Joint,Padangnese Restaurant,Nightclub,Noodle House,Office
8,Cipete Utara,Post Office,Bakery,Snack Place,Diner,Juice Bar,Karaoke Bar,Nightclub,Noodle House,Japanese Restaurant,Office
9,Cipulir,Bakery,Javanese Restaurant,Pizza Place,Restaurant,Coffee Shop,Seafood Restaurant,Chinese Restaurant,Food Truck,Furniture / Home Store,Donut Shop


# Adding Population Data of each Neighborhood / Kelurahan into the dataset

In [28]:
dfpop = pd.read_csv('data-kependudukan-tahun-2019.csv')
dfpop.head()

,kecamatan,kelurahan,jumlah_lakilaki,jumlah_perempuan
0,TEBET,TEBET TIMUR,10473,11079
1,TEBET,TEBET BARAT,12505,12987
2,TEBET,MENTENG DALAM,22440,22481
3,TEBET,KEBON BARU,21562,21493
4,TEBET,BUKIT DURI,21103,20812


In [29]:
dfpop['kecamatan'] = dfpop['kecamatan'].str.title()
dfpop['kelurahan'] = dfpop['kelurahan'].str.title()
dfpop.head()

,kecamatan,kelurahan,jumlah_lakilaki,jumlah_perempuan
0,Tebet,Tebet Timur,10473,11079
1,Tebet,Tebet Barat,12505,12987
2,Tebet,Menteng Dalam,22440,22481
3,Tebet,Kebon Baru,21562,21493
4,Tebet,Bukit Duri,21103,20812


*Now as data is formatted, both kelurahan and neighborhood of dfpop and neighborhoods_venues_sorted is already formatted, we can proceed to concat the data!*

In [30]:
tempman = []
tempwom = []
for x in neighborhoods_venues_sorted['Neighborhood']:
    tempman.append(dfpop[dfpop['kelurahan'] == x]['jumlah_lakilaki'].values)
    tempwom.append(dfpop[dfpop['kelurahan'] == x]['jumlah_perempuan'].values)

In [31]:
popman = pd.DataFrame(tempman,columns=['Man Population'])
popwom = pd.DataFrame(tempwom,columns=['Woman Population'])


In [32]:
neighborhoods_venues_sorted = pd.concat([popman,popwom,neighborhoods_venues_sorted],axis=1)

neighborhoods_venues_sorted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Man Population          59 non-null     float64
 1   Woman Population        59 non-null     float64
 2   Neighborhood            61 non-null     object 
 3   1st Most Common Venue   61 non-null     object 
 4   2nd Most Common Venue   61 non-null     object 
 5   3rd Most Common Venue   61 non-null     object 
 6   4th Most Common Venue   61 non-null     object 
 7   5th Most Common Venue   61 non-null     object 
 8   6th Most Common Venue   61 non-null     object 
 9   7th Most Common Venue   61 non-null     object 
 10  8th Most Common Venue   61 non-null     object 
 11  9th Most Common Venue   61 non-null     object 
 12  10th Most Common Venue  61 non-null     object 
dtypes: float64(2), object(11)
memory usage: 6.3+ KB


In [33]:
neighborhoods_venues_sorted.head()

,Man Population,Woman Population,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,13218.0,12819.0,Bangka,Fast Food Restaurant,Indonesian Restaurant,Bakery,Pizza Place,Optical Shop,Music School,Music Store,Music Venue,Nightclub,Noodle House
1,31859.0,31997.0,Bintaro,Fast Food Restaurant,Indonesian Restaurant,Convenience Store,Coffee Shop,Fish & Chips Shop,Asian Restaurant,Gym,Bakery,Sandwich Place,Supermarket
2,23167.0,22954.0,Ciganjur,Restaurant,Soccer Field,Convenience Store,Café,Wings Joint,Padangnese Restaurant,Music Venue,Nightclub,Noodle House,Office
3,6561.0,6467.0,Cikoko,Fast Food Restaurant,Convenience Store,Coffee Shop,Gas Station,Japanese Restaurant,Asian Restaurant,Supermarket,Café,Chinese Restaurant,Padangnese Restaurant
4,15976.0,15829.0,Cilandak Barat,Coffee Shop,Café,Asian Restaurant,Indonesian Restaurant,Pizza Place,Fast Food Restaurant,Ice Cream Shop,Bookstore,Bakery,Thai Restaurant


## Clustering our neighborhood

running k means to cluster neighborhood into 5 cluster

In [34]:
kclusters = 4

jaksel_grouped_clustering = jaksel_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(jaksel_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 1, 0, 0, 0, 3, 0, 0, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [35]:
neighborhoods_venues_sorted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Man Population          59 non-null     float64
 1   Woman Population        59 non-null     float64
 2   Neighborhood            61 non-null     object 
 3   1st Most Common Venue   61 non-null     object 
 4   2nd Most Common Venue   61 non-null     object 
 5   3rd Most Common Venue   61 non-null     object 
 6   4th Most Common Venue   61 non-null     object 
 7   5th Most Common Venue   61 non-null     object 
 8   6th Most Common Venue   61 non-null     object 
 9   7th Most Common Venue   61 non-null     object 
 10  8th Most Common Venue   61 non-null     object 
 11  9th Most Common Venue   61 non-null     object 
 12  10th Most Common Venue  61 non-null     object 
dtypes: float64(2), object(11)
memory usage: 6.3+ KB


In [37]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)




ValueError: cannot insert Cluster Labels, already exists

In [48]:
jaksel_merged = df

In [49]:
jaksel_merged.head()

,Kode Pos,Kecamatan,Kelurahan,Latitude,Longitude
0,12430,Cilandak,Cilandak Barat,-6.290266,106.796215
1,12410,Cilandak,Cipete Selatan,-6.272540,106.805768
2,12420,Cilandak,Gandaria Selatan,-6.271562,106.795172
3,12440,Cilandak,Lebak Bulus,-6.302568,106.780354
4,12450,Cilandak,Pondok Labu,-6.309144,106.797938


In [50]:
jaksel_merged.rename(columns ={'Kode Pos':'Postal Codes','Kecamatan':'Borough','Kelurahan':'Neighborhood'}, inplace = True)

In [52]:
# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
jaksel_merged = jaksel_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

jaksel_merged.head() # check the last columns!

,Postal Codes,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Man Population,Woman Population,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,12430,Cilandak,Cilandak Barat,-6.290266,106.796215,0.0,15976.0,15829.0,Coffee Shop,Café,Asian Restaurant,Indonesian Restaurant,Pizza Place,Fast Food Restaurant,Ice Cream Shop,Bookstore,Bakery,Thai Restaurant
1,12410,Cilandak,Cipete Selatan,-6.272540,106.805768,0.0,13040.0,13162.0,Moving Target,Javanese Restaurant,Arcade,Food Court,Café,Wings Joint,Padangnese Restaurant,Nightclub,Noodle House,Office
2,12420,Cilandak,Gandaria Selatan,-6.271562,106.795172,3.0,26342.0,26603.0,Food Truck,Indonesian Restaurant,Multiplex,Fried Chicken Joint,Convenience Store,Café,Wings Joint,Optical Shop,Nightclub,Noodle House
3,12440,Cilandak,Lebak Bulus,-6.302568,106.780354,0.0,29928.0,29878.0,Convenience Store,Indonesian Restaurant,Café,Bubble Tea Shop,Asian Restaurant,Shabu-Shabu Restaurant,Paper / Office Supplies Store,Coffee Shop,Supermarket,Soup Place
4,12450,Cilandak,Pondok Labu,-6.309144,106.797938,2.0,21515.0,21618.0,Grocery Store,Chinese Restaurant,Multiplex,Music Store,Music Venue,Nightclub,Noodle House,Office,Optical Shop,Padangnese Restaurant


In [53]:
jaksel_merged.dropna(inplace=True)

Finally, let's visualize the resulting clusters

In [54]:
import folium

In [56]:
# create map
map_clusters = folium.Map(location=[latjaksel,longjaksel], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(jaksel_merged['Latitude'], jaksel_merged['Longitude'], jaksel_merged['Neighborhood'], jaksel_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

lets examine the cluster

In [57]:
jaksel_merged.loc[jaksel_merged['Cluster Labels'] == 0, jaksel_merged.columns[[1] + list(range(5, jaksel_merged.shape[1]))]]

,Borough,Cluster Labels,Man Population,Woman Population,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Cilandak,0.0,15976.0,15829.0,Coffee Shop,Café,Asian Restaurant,Indonesian Restaurant,Pizza Place,Fast Food Restaurant,Ice Cream Shop,Bookstore,Bakery,Thai Restaurant
1,Cilandak,0.0,13040.0,13162.0,Moving Target,Javanese Restaurant,Arcade,Food Court,Café,Wings Joint,Padangnese Restaurant,Nightclub,Noodle House,Office
3,Cilandak,0.0,29928.0,29878.0,Convenience Store,Indonesian Restaurant,Café,Bubble Tea Shop,Asian Restaurant,Shabu-Shabu Restaurant,Paper / Office Supplies Store,Coffee Shop,Supermarket,Soup Place
9,Jagakarsa,0.0,35441.0,34704.0,College Residence Hall,Light Rail Station,Theme Park,Wings Joint,Padangnese Restaurant,Music Store,Music Venue,Nightclub,Noodle House,Office
11,Kebayoran Baru,0.0,21059.0,20819.0,Post Office,Bakery,Snack Place,Diner,Juice Bar,Karaoke Bar,Nightclub,Noodle House,Japanese Restaurant,Office
13,Kebayoran Baru,0.0,5602.0,5661.0,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Hotel,Convenience Store,Restaurant,Pizza Place,Padangnese Restaurant,Buffet,Bubble Tea Shop
14,Kebayoran Baru,0.0,8939.0,8683.0,Indonesian Restaurant,Coffee Shop,Food Truck,Convenience Store,Café,Noodle House,Steakhouse,Restaurant,Soup Place,Japanese Restaurant
15,Kebayoran Baru,0.0,1533.0,1668.0,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Hotel,Convenience Store,Restaurant,Pizza Place,Padangnese Restaurant,Buffet,Bubble Tea Shop
16,Kebayoran Baru,0.0,7048.0,6993.0,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Hotel,Convenience Store,Restaurant,Pizza Place,Padangnese Restaurant,Buffet,Bubble Tea Shop
17,Kebayoran Baru,0.0,3204.0,3572.0,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Hotel,Convenience Store,Restaurant,Pizza Place,Padangnese Restaurant,Buffet,Bubble Tea Shop


the first cluster has interest in parks, swimming pool, and yoga studio !

In [58]:
jaksel_merged.loc[jaksel_merged['Cluster Labels'] == 1, jaksel_merged.columns[[1] + list(range(5, jaksel_merged.shape[1]))]]

,Borough,Cluster Labels,Man Population,Woman Population,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Jagakarsa,1.0,23167.0,22954.0,Restaurant,Soccer Field,Convenience Store,Café,Wings Joint,Padangnese Restaurant,Music Venue,Nightclub,Noodle House,Office
7,Jagakarsa,1.0,37643.0,37141.0,Convenience Store,Garden,Chinese Restaurant,Acehnese Restaurant,Pharmacy,Music Venue,Nightclub,Noodle House,Office,Optical Shop
30,Mampang Prapatan,1.0,26711.0,26324.0,Convenience Store,Airport,Restaurant,Boutique,Hobby Shop,Wings Joint,Paper / Office Supplies Store,Music Venue,Nightclub,Noodle House
33,Pancoran,1.0,17291.0,16856.0,Convenience Store,Clothing Store,Movie Theater,Bookstore,Café,Sushi Restaurant,Spa,Coffee Shop,Wings Joint,Nightclub
42,Pasar Minggu,1.0,21364.0,21331.0,Convenience Store,Bridal Shop,Asian Restaurant,Food Truck,Paper / Office Supplies Store,Music Venue,Nightclub,Noodle House,Office,Optical Shop
44,Pasar Minggu,1.0,23288.0,22627.0,Convenience Store,Bus Line,Restaurant,Asian Restaurant,Café,Food Court,Wings Joint,Paper / Office Supplies Store,Noodle House,Office


the second clusters has interest in cofee shops and cafes/restaurant!

In [59]:
jaksel_merged.loc[jaksel_merged['Cluster Labels'] == 2, jaksel_merged.columns[[1] + list(range(5, jaksel_merged.shape[1]))]]

,Borough,Cluster Labels,Man Population,Woman Population,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Cilandak,2.0,21515.0,21618.0,Grocery Store,Chinese Restaurant,Multiplex,Music Store,Music Venue,Nightclub,Noodle House,Office,Optical Shop,Padangnese Restaurant


third clusters has interest in printing shops

In [60]:
jaksel_merged.loc[jaksel_merged['Cluster Labels'] == 3, jaksel_merged.columns[[1] + list(range(5, jaksel_merged.shape[1]))]]

,Borough,Cluster Labels,Man Population,Woman Population,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Cilandak,3.0,26342.0,26603.0,Food Truck,Indonesian Restaurant,Multiplex,Fried Chicken Joint,Convenience Store,Café,Wings Joint,Optical Shop,Nightclub,Noodle House
6,Jagakarsa,3.0,23010.0,22862.0,Food,Department Store,Grocery Store,Farmers Market,Fast Food Restaurant,Wings Joint,Nightclub,Noodle House,Office,Optical Shop
8,Jagakarsa,3.0,32850.0,32212.0,Acehnese Restaurant,Steakhouse,Noodle House,Convenience Store,Fast Food Restaurant,Food Truck,Wings Joint,Padangnese Restaurant,Music Venue,Nightclub
10,Jagakarsa,3.0,24031.0,23901.0,Indonesian Restaurant,Javanese Restaurant,Auto Workshop,Light Rail Station,Fast Food Restaurant,Padangnese Restaurant,Music Store,Music Venue,Nightclub,Noodle House
12,Kebayoran Baru,3.0,23978.0,24322.0,Indonesian Restaurant,Food Truck,Convenience Store,Grocery Store,Steakhouse,Chinese Restaurant,Supermarket,Sushi Restaurant,Betawinese Restaurant,Javanese Restaurant
27,Mampang Prapatan,3.0,13218.0,12819.0,Fast Food Restaurant,Indonesian Restaurant,Bakery,Pizza Place,Optical Shop,Music School,Music Store,Music Venue,Nightclub,Noodle House
34,Pancoran,3.0,25202.0,24869.0,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Food Truck,Padangnese Restaurant,Music Store,Music Venue,Nightclub,Noodle House,Office,Optical Shop
45,Pesanggrahan,3.0,31859.0,31997.0,Fast Food Restaurant,Indonesian Restaurant,Convenience Store,Coffee Shop,Fish & Chips Shop,Asian Restaurant,Gym,Bakery,Sandwich Place,Supermarket
46,Pesanggrahan,3.0,16948.0,16773.0,Fast Food Restaurant,Arcade,Food Truck,Salad Place,Paper / Office Supplies Store,Music Venue,Nightclub,Noodle House,Office,Optical Shop
62,Tebet,3.0,22440.0,22481.0,Indonesian Restaurant,Hotel,Food Truck,Tennis Court,Grocery Store,Fast Food Restaurant,Coffee Shop,Padangnese Restaurant,Music Venue,Nightclub


From the clusters we see that the first cluster offers varieties of restaurants and is near to college residence
it allows FOODINC to gain market shares of due to similiar neighborhoods. 

as we see that we need higher demand and lower supplies, and we can see that at borough of Pasar Minggu,at one of its neighborhood it has high population and demand for fast food.

we can also conclude due to college residences in the same cluster, we can make package deals for college students to buy within their pocket money

with that in mind we can conclode that Pasar Minggu is the best place to put a new chain of fast food restaurant!